#### DTUMOS 0단계 
- 시뮬레이션을 하는 데이터를 시뮬레이션 module에 맞게 정제하며, 본인이 원하는 분석 시간 범위를 결정한다.

In [1]:
import pandas as pd 
from shapely.geometry import Point

taxi_raw_data = pd.read_csv('./data/raw_data/train.csv')

essential_columns = ['id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
taxi_raw_data = taxi_raw_data[essential_columns]

taxi_raw_data = taxi_raw_data.rename(columns={'id':'taxi_id', 'pickup_datetime':'ride_time','dropoff_datetime':'alight_time',
                                              'pickup_longitude':'ride_lon', 'pickup_latitude':'ride_lat', 'dropoff_longitude':'alight_lon', 
                                              'dropoff_latitude':'alight_lat'})

# 문자열로 되어 있는 시간을 datetime type으로 변경
taxi_raw_data['ride_time'] = pd.to_datetime(taxi_raw_data['ride_time'], format='%Y-%m-%d %H:%M:%S')
taxi_raw_data['alight_time'] = pd.to_datetime(taxi_raw_data['alight_time'], format='%Y-%m-%d %H:%M:%S')

# 분석 타겟 시간은 2016-04-08 
taxi_raw_data = taxi_raw_data.loc[(taxi_raw_data['ride_time'] >= pd.Timestamp(2016,4,8)) & (taxi_raw_data['alight_time'] < pd.Timestamp(2016,4,9))]

taxi_raw_data['ride_geometry'] = [Point(i[0],i[1]) for i in taxi_raw_data[['ride_lon', 'ride_lat']].values]
taxi_raw_data['alight_geometry'] = [Point(i[0],i[1]) for i in taxi_raw_data[['alight_lon', 'alight_lat']].values]

/home/yh_zoo/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/home/yh_zoo/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


---
#### DTUMOS 1단계
- convert_data.py
    - generate_random_location.py

In [2]:
from module.convert_data import generate_simulation_base_data

passenger, taxi, YMD = generate_simulation_base_data(taxi_raw_data, target_place='New York County, 뉴욕, 미국')

/home/yh_zoo/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


---

In [3]:
from module.dispatch_helper import *
from module.dashboard_plot import *
import pickle

result = dispatch_module(passenger, taxi, YMD, dispatch_mode='OSRM')

#all data save
result[2].extend([passenger, taxi, YMD])

with open('./data/result_data/all_data/result.pickle', 'wb') as f:
    pickle.dump(result, f)

100%|██████████| 1440/1440 [14:11<00:00,  1.69it/s]


In [1]:
from module.dashboard_plot import *
import pickle

with open('./data/result_data/all_data/result.pickle', 'rb') as f:
    result = pickle.load(f)
    
make_dashboard_plot(result, path='./data/result_data/dashboard_plot/')

import json
import pickle

### simulation data save
trips_simulation_data, passenger_simulation_data , empty_taxi_simulation_data = result[0]

# trip
with open(f'./data/result_data/simulation_data/trip.json', 'w') as f:
    json.dump(trips_simulation_data,f)

# passenger
with open(f'./data/result_data/simulation_data/passenger.json', 'w') as f:
    json.dump(passenger_simulation_data,f)

# empty_taxi
with open(f'./data/result_data/simulation_data/empty_taxi.json', 'w') as f:
    json.dump(empty_taxi_simulation_data,f)

/home/yh_zoo/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
